**[SQL Micro-Course Home Page](https://www.kaggle.com/learn/SQL)**

---


# Introduction

You've built up your SQL skills enough that the remaining hands-on exercises will use different datasets than you see in the explanations. If you need to familiarize with a new dataset, you can run a couple of `SELECT` queries to extract and review the data you need. 

The next exercises are also more challenging than what you've done so far. Don't worry, you are ready for it.

Run the code in the following cell to get everything set up:

In [1]:
# Set up feedack system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex4 import *

# import package with helper functions 
import bq_helper

# create a helper object for this dataset
education_data = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                          dataset_name="world_bank_intl_education")

Using Kaggle's public dataset BigQuery integration.


Then write the code to answer the questions below.

# Questions

## 1) Which countries spend the largest fraction of GDP on education?

The World Bank has made tons of interesting education data available through BigQuery. Run the following cell to see the first few rows of the table.

In [2]:
education_data.head('international_education')

,country_name,country_code,indicator_name,indicator_code,year,value
0,Angola,AGO,"GDP per capita, PPP (constant 2011 internation...",NY.GDP.PCAP.PP.KD,1992,3.793646e+03
1,Turkey,TUR,"Population, ages 7-12, total",SP.POP.0712.TO.UN,2000,8.194184e+06
2,Burkina Faso,BFA,Africa Dataset: Average number of pupils per r...,UIS.AFR.PBR.1.G3.PU.READ,2010,6.214100e-01
3,Chile,CHL,Adjusted net intake rate to Grade 1 of primary...,UIS.NIRA.1.M,2010,8.187383e+01
4,Sub-Saharan Africa,SSF,Percentage of students in upper secondary gene...,UIS.FEP.3.GPV,2010,4.475259e+01


The indicator code describes what type of data is shown in a given row.  

One interesting indicator code is `SE.XPD.TOTL.GD.ZS`. The name says this is `Government expenditure on education as % of GDP (%)`.

Write a query to get a dataframe returning **average value of these rows for each country in the dataset between the years 2010-2017** (including 2010 and 2017 in the average). 

Requirements:
- Your results should have the country name rather than the country code. You will have one row for each country.
- The aggregation for an average is **AVG**.  Use the name `avg_ed_spending_pct` for the column created by this aggregation.
- Order the results so the countries that spend the largest fraction of GDP on education show up first.

In case it's useful to see a sample query, here's a query you saw in the tutorial (using a different database):
```
# query to find how many accidents happen each day of the week
query = """SELECT COUNT(consecutive_number) num_accidents, 
                  EXTRACT(DAYOFWEEK FROM timestamp_of_crash)
            FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2016`
            GROUP BY EXTRACT(DAYOFWEEK FROM timestamp_of_crash)
            ORDER BY COUNT(consecutive_number) DESC
        """
accidents_by_day = accidents.query_to_pandas_safe(query)
```

In [3]:
# Your Code Here

country_spend_pct_query = """
SELECT country_name, AVG(value) avg_ed_spending_pct
FROM `bigquery-public-data.world_bank_intl_education.international_education`
WHERE indicator_code = 'SE.XPD.TOTL.GD.ZS' and year >= 2010 and year <= 2017
GROUP BY country_name
ORDER BY avg_ed_spending_pct DESC
"""

country_spending_results = education_data.query_to_pandas_safe(country_spend_pct_query)

print(country_spending_results.head())
print(country_spending_results.loc[country_spending_results['country_name'] == 'Japan'])
#q_1.check()

            country_name  avg_ed_spending_pct
0                   Cuba            12.837270
1  Micronesia, Fed. Sts.            12.467750
2        Solomon Islands            10.001080
3                Moldova             8.372153
4                Namibia             8.349610
    country_name  avg_ed_spending_pct
106        Japan             3.646636


In [4]:
q_1.hint()
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> The part before `FROM` should be `SELECT country_name, AVG(value) avg_ed_spending_pct`

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

SELECT country_name, AVG(value) avg_ed_spending_pct
FROM `bigquery-public-data.world_bank_intl_education.international_education`
WHERE indicator_code = 'SE.XPD.TOTL.GD.ZS' and year >= 2010 and year <= 2017
GROUP BY country_name
ORDER BY avg_ed_spending_pct DESC

```

## 2) Identifying Interesting Codes to Explore
The last question started by telling you to focus on rows with the code `SE.XPD.TOTL.GD.ZS`. But how would you find interesting indicator codes to explore?

There are 1000s of codes in the dataset, so it would time consuming to review them all. But many codes are available for only a few countries. When browsing the options for different codes, you might restrict yourself to codes that are reported by many countries.

Write the query below that selects the indicator code and indicator name for all codes with at least 175 rows in the year 2016.

Requirements:
- One row in your results per code
- The columns in your results should be called `indicator_code`, `indicator_name` and `num_rows`.
- Only select codes with 175 or more rows in the raw database (exactly 175 rows would be included).
- To get both the `indicator_code` and `indicator_name` in your resulting DataFrame, you need to include both in your **SELECT** statement (in addition to a **COUNT** aggregation). This in turn requires you to include both in your **GROUP BY** clause.
- Order results from most frequent to least frequent.

In [5]:
# Your Code Here
code_count_query = """
                    SELECT indicator_code, indicator_name, COUNT(indicator_code) num_rows
                    FROM `bigquery-public-data.world_bank_intl_education.international_education`
                    WHERE year = 2016
                    GROUP BY indicator_code, indicator_name
                    HAVING COUNT(indicator_code) > 175
                    ORDER BY COUNT(indicator_code) DESC
                    
"""

code_count_results = education_data.query_to_pandas_safe(code_count_query)

print(code_count_results.head())
q_2.check()

      indicator_code                   indicator_name  num_rows
0        SP.POP.TOTL                Population, total       232
1        SP.POP.GROW     Population growth (annual %)       232
2     IT.NET.USER.P2  Internet users (per 100 people)       223
3  SP.POP.TOTL.FE.ZS  Population, female (% of total)       213
4     SP.POP.1564.TO    Population, ages 15-64, total       213


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
q_2.hint()
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> The part before `FROM` is `SELECT indicator_code, indicator_name, COUNT(1) AS num_rows`

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

code_count_query = """SELECT indicator_code, indicator_name, COUNT(1) AS num_rows
            FROM `bigquery-public-data.world_bank_intl_education.international_education`
            WHERE year = 2016
            GROUP BY indicator_name, indicator_code
            HAVING COUNT(1) >= 175
            ORDER by COUNT(1) DESC
        """

```

---
# Keep Going
**[Click here](https://www.kaggle.com/dansbecker/as-with)** to learn how *WITH-AS* clauses  can clean up your code and help you construct more complex queries.


---
**[SQL Micro-Course Home Page](https://www.kaggle.com/learn/SQL)**

